## Basic sqoop import Command

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--target-dir sqoop_import_dir

1. Target-dir must not exist beforehand. If it exists, the command while fail with the same error. 
2. This command will create the directory while sqooping, then would put the table data inside this directory as part files. 
3. There would be no subfolder created inside target-dir with the table name, datafiles would be put directly inside target-dir.

If target-dir is not provided, sqoop will create a directory in the current working directory with the same name as table name.

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user

## Controlling Parellelism

1. To leverage parallelism, we need to provide number of mappers in the import command. There would be that many parallel imports executed, as mentioned in the --num-mappers parameter. 
2. That many number of part files would get created as there are number of mappers mentioned in the sqoop import command

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--target-dir sqoop_import_dir \
--num-mappers 3


3. When performing parallel imports, Sqoop needs a criterion by which it can split the workload. Sqoop uses a splitting column to split the workload. By default, Sqoop will identify the primary key column (if present) in a table and use it as the splitting column. The low and high values for the splitting column are retrieved from the database, and the map tasks operate on evenly-sized components of the total range. For example, if you had a table with a primary key column of id whose minimum value was 0 and maximum value was 1000, and Sqoop was directed to use 4 tasks, Sqoop would run four processes which each execute SQL statements of the form ```SELECT * FROM sometable WHERE id >= lo AND id < hi```, with (lo, hi) set to (0, 250), (250, 500), (500, 750), and (750, 1001) in the different tasks. **(In the above sqoop, user_id is the primary key of the table 'user', which has been used as splitting key as can be seen from the sqoop log printed on console)**

4. If the actual values for the primary key are not uniformly distributed across its range, then this can result in unbalanced tasks. You should explicitly choose a different column with the --split-by argument. For example, --split-by employee_id. This is also applicable for sqooping from a table which does not have a primary key, hence split-by column has to be mentioned **(Below we are sqooping from a mysql table 'logs' which had the columns logid,logdate,jobname,stepname,status,error_message. None of these has been made primary key of the table. Hence we have to either mention a column with --split-by parameter, or perform a sequential import using '-m 1'. Else it will throw error -** *ERROR tool.ImportTool: Error during import: No primary key could be found for table logs. Please specify one with --split-by or perform a sequential import with '-m 1'.*

5. The split-by column has to be an integer column. If a textual column is passed, it would throw error - *ERROR tool.ImportTool: Encountered IOException running import job: java.io.IOException: Generating splits for a textual index column allowed only
 in case of "-Dorg.apache.sqoop.splitter.allow_text_splitter=true" property passed as a parameter* 

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table logs \
--target-dir sqoop_import_dir \
--num-mappers 3 \
--split-by logid


### Warehouse-dir parameter

**1. warehouse-dir may exist beforehand, if does not exist it will be created while sqooping**
2. During sqooping, a subfolder would be created inside the warehouse-dir with same name as the table name
3. Data would be put inside this subfolder as part files

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--warehouse-dir sqoop_import_dir